<a href="https://colab.research.google.com/github/Necl0/HTNDoodleProject/blob/main/Doodl_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quickdraw
!pip install keras

In [85]:
# Import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense
from keras.applications.vgg19 import VGG19
from quickdraw import QuickDrawDataGroup

In [86]:
max_d = 1001

airplanes = QuickDrawDataGroup('airplane', max_drawings=max_d)
castle = QuickDrawDataGroup('castle', max_drawings=max_d)
dragon = QuickDrawDataGroup('dragon', max_drawings=max_d)
duck = QuickDrawDataGroup('duck', max_drawings=max_d)
fork = QuickDrawDataGroup('fork', max_drawings=max_d)
hexagon = QuickDrawDataGroup('hexagon', max_drawings=max_d)
key = QuickDrawDataGroup('key', max_drawings=max_d)
mountain = QuickDrawDataGroup('mountain', max_drawings=max_d)
pizza = QuickDrawDataGroup('pizza', max_drawings=max_d)
star = QuickDrawDataGroup('star', max_drawings=max_d)
sun = QuickDrawDataGroup('sun', max_drawings=max_d)
tree = QuickDrawDataGroup('tree', max_drawings=max_d)

# Define a class-to-number mapping
class_mapping = {'airplane': 0, 'castle': 1, 'dragon': 2, 'duck': 3, 'fork': 4,
                 'hexagon': 5, 'key': 6, 'mountain': 7, 'pizza': 8, 'star': 9, 'sun': 10, 'tree': 11}

# Map class labels to numeric values
img_classes = np.array([[class_mapping[drawing.name] for drawing in c.drawings] for c in [airplanes, castle, dragon, duck, fork,
                                                                                          hexagon, key, mountain, pizza,
                                                                                          star, sun, tree]]).flatten()[:-12]

img_data = np.array([[np.array(drawing.get_image().resize((224, 224)).convert('L')) for drawing in c.drawings] for c in
                     [airplanes, castle, dragon,
                      duck, fork, hexagon,
                      key, mountain,
                      pizza, star,
                      sun, tree]]
                    )

img_data = img_data.reshape((img_data.shape[0] * img_data.shape[1],) + img_data.shape[2:])
print("Data loaded!")

loading airplane drawings
load complete
loading castle drawings
load complete
loading dragon drawings
load complete
loading duck drawings
load complete
loading fork drawings
load complete
loading hexagon drawings
load complete
loading key drawings
load complete
loading mountain drawings
load complete
loading pizza drawings
load complete
loading star drawings
load complete
loading sun drawings
load complete
loading tree drawings
load complete
Data loaded!


In [87]:
print(img_classes.shape)
print(img_data.shape)

(12000,)
(12000, 224, 224)


In [88]:
from sklearn.model_selection import train_test_split

# split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(
    img_data,
    img_classes,
    test_size=0.2,
    random_state=42,
    shuffle=True
)


In [89]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(9600, 224, 224) (2400, 224, 224) (9600,) (2400,)


In [90]:
# Add an extra dimension of size 3 to each array
x_train = np.expand_dims(x_train, axis=-1)
x_train = np.repeat(x_train, 3, axis=-1)

x_test = np.expand_dims(x_test, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)


print("Updated shapes:")
print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

Updated shapes:
x_train: (9600, 224, 224, 3)
x_test: (2400, 224, 224, 3)
y_train: (9600,)
y_test: (2400,)


In [91]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Load VGG19 model without top layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(12, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [92]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(9600, 224, 224, 3) (2400, 224, 224, 3) (9600,) (2400,)


In [93]:
# Train the model
batch_size = 32
epochs = 20

model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test)
)

Epoch 1/20
300/300 [==============================] - 28s 58ms/step - loss: 0.7162 - accuracy: 0.8097 - val_loss: 0.5364 - val_accuracy: 0.8575
Epoch 2/20
300/300 [==============================] - 16s 55ms/step - loss: 0.3909 - accuracy: 0.8886 - val_loss: 0.5230 - val_accuracy: 0.8554
Epoch 3/20
300/300 [==============================] - 16s 55ms/step - loss: 0.2946 - accuracy: 0.9136 - val_loss: 0.4905 - val_accuracy: 0.8679
Epoch 4/20
300/300 [==============================] - 16s 55ms/step - loss: 0.2413 - accuracy: 0.9297 - val_loss: 0.5335 - val_accuracy: 0.8612
Epoch 5/20
300/300 [==============================] - 16s 55ms/step - loss: 0.2002 - accuracy: 0.9397 - val_loss: 0.5072 - val_accuracy: 0.8779
Epoch 6/20
300/300 [==============================] - 16s 55ms/step - loss: 0.1581 - accuracy: 0.9485 - val_loss: 0.5383 - val_accuracy: 0.8625
Epoch 7/20
300/300 [==============================] - 17s 55ms/step - loss: 0.1373 - accuracy: 0.9529 - val_loss: 0.5781 - val_accuracy:

In [97]:
import pickle
with open('HTNDoodl.pkl', 'wb') as file:
    pickle.dump({'model': model}, file)